## Prepare Data

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#Asses data

In [9]:
#Asses raw data files and apply function above to begin cleaning the data
#Year and Go-round could be inputs or pulled from the file string to be 
df = pd.read_csv("PB-Futurity-2020-1st-Go.csv",  header = None)
df.head()

0                                                  1  \
0    #      Horse\rAge, Color, Sex\rSire\rDam, Dam's Sire   
1   13  VF Cream Rises\r2016 Sorrel Mare\rEddie Stinso...   
2  186  KN Snap Back\r2016 Mare\rEddie Stinson\rKN Han...   
3  163  The Right Version\r2015 Gray Mare\rWinners Ver...   
4   18  Makana\r2015 Sorrel Mare\rSlick By Design\rRod...   

                                                   2  \
0                                      Owner / Rider   
1  Ademir Jose Rorato\r\rLexington, OK\r\rKelsey ...   
2  Julien Veileux\r\rSt. Alfred, Quebec\r\rCaroli...   
3   Joe Hadley\r\rPlain City, UT\r\rMcKinlee Kellett   
4    Michele McLeod\r\rOntario, OR\r\rLindsey McLeod   

                                                   3    4    5    6    7  \
0                                            Breeder  NaN  NaN  NaN  NaN   
1                         Victory Farms\r\rA\rda, OK  NaN  NaN    O  NaN   
2                  Stephen Nicholes\r\rBrownwood, TX  NaN  NaN    O  NaN   
3                 Joe Hadley\r\rP  l a  i n City, UT  NaN  NaN    O  NaN   
4  C  h   a  r l i e    C  o  l e    &    J a  s ...  NaN  NaN    O  NaN   

        8  
0  1st Go  
1  17.067  
2  17.070  
3  17.102  
4  17.125

#### Create a function to create tidy data by ensuring each type of "observation" has its own column, for each year since result formatting varied.  

In [10]:
def expandcolumns_goround(file):
    '''
    INPUT string of path to CSV file converted from PDF of go round results from Pink Buckle Futurity
    Futurity result formatting varies slightly year to year. 
    OUTPUT dataframe containing information relative to a stallion's offspring performance
    '''
    df = pd.read_csv(file,  header = None)
    f = file
    fvals = f.split(sep = '.')
    fvals = fvals[0]
    fvals=fvals.split(sep = '-')
    sex_list = ['Gelding','Mare','Stallion','M','G','S']
    sex_dict = {'G)':'Gelding','M)':'Mare','S)':'Stallion', ' M)':'Mare', 'G':'Gelding'}
    if fvals[2] == '2020':
    #break column 2 into its own data frame, then create tidy data by ensuring each type of "observation" has its own column
        col2 = df.iloc[:,1]
        col2 = col2.str.split (pat = '\r',expand = True)
        dds = (col2.iloc[:,3]).str.split(pat=', ',expand = True)
        acs = ((col2.iloc[:,1]).str.split(pat=' ',expand = True))

        col2 = df.iloc[:,1]
        col2 = col2.str.split (pat = '\r',expand = True)
        dds = (col2.iloc[:,3]).str.split(pat=', ',expand = True)
        dds_header = dds.iloc[0]# Save first row for headers
        dds = dds[1:]
        dds.columns = dds_header
        acs = ((col2.iloc[:,1]).str.split(pat=' ',expand = True))
        acs_header = acs.iloc[0]# acs.iloc[0]# Save first row for headers
        acs.columns = acs_header
        acs=acs[1:]
        # remove columns that have been expaned and concatenate dataframes
        col2 = col2.drop(columns = [1,3])
        col2_header = col2.iloc[0]# Save first row for headers
        col2.columns = col2_header
        col2 = col2[1:]
         # expand column 3 in df the same way as 2 but assign header values
        col3 = (df.iloc[:,2]).str.split(pat='\r', expand = True)

        col3_header = ['Owner1','Owner1_location','Owner2','Owner2_location', 'Rider']
        col3.columns = col3_header
        col3=col3[1:]
        col3=col3['Rider']# We are only using this information for the analysis

        #col4 = (df.iloc[:,3]).str.split(pat='\r', expand = True)
       # col4_header = col4.iloc[0]
       # col4.columns = ['Breeder', 'overflow', 'BreederLoc1', 'BreederLoc1', 'overflow2'] #considering adding overflow to the original columns. IDK how that is going to work. 
       # col4=col4[1:]
        #drop the columns that have been expanded & those not used
        df2 = df.drop(columns = [1,2,3,4,5,6,7])
        df2_header = df2.iloc[0] #promote the first row to headers 
        df2= df2[1:]
        df2.columns = df2_header
        df = pd.concat([col2,dds,acs,col3,df2], axis=1)#concatenate the expanded data
        df['Sex'] = np.where(df.iloc[:,7].notnull(), df.iloc[:,7],df['Sex'])#some of the sex information got pushed into another column using the " " as a delimiter. This moves it back
        #Some entrants did not list color and the sex information was stored in the color field
       
        df['Sex']=np.where(df['Color,'].isin(sex_list), df['Color,'], df['Sex'])
        df = df.rename(columns = {'Color,':'Color'})
        df = df.rename(columns = {'Age,':'Age'})

    elif fvals[2] == '2018' :
        col2 = df.iloc[:,1]
        col2 = col2.str.split (pat = '\r',expand = True)
        horse_age_color_sex = col2.iloc[:,0].str.split(pat = '(', expand = True)
        col2 = col2.drop(columns = [0])
        age_color_sex = horse_age_color_sex.iloc[:,1].str.split(pat='‐', expand = True)
        age_color_sex = age_color_sex[1:]
        horse = horse_age_color_sex.drop(columns = 1)
        horse_head = ['Horse']
        horse=horse[1:]
        horse.columns = horse_head
        acs_header = ['Age', 'Color', 'Sex']
        age_color_sex = age_color_sex.drop(columns = 3)
        age_color_sex.columns = acs_header
        #replace any erroneous sex data that is be in the color column
       # age_color_sex['Sex']=np.where(age_color_sex['Color'].isin(sex_list), age_color_sex['Color'], age_color_sex['Sex'])
        sex_dict = {'G)':'Gelding','M)':'Mare','S)':'Stallion', ' M)':'Mare', 'Bay':'Gelding'} # replace abbreviation with word and brute force change one erroneous point of Bay
        age_color_sex = age_color_sex.replace(to_replace = sex_dict)
        col2_header = col2.iloc[0]# Save first row for headers
        col2.columns = col2_header
        col2 = col2[1:]
       
        col3 = (df.iloc[:,3]).str.split(pat='\r', expand = True)
        col3=col3.drop(columns = 1)
        col3_header = ['Rider']
        col3.columns = col3_header
        col3=col3[1:]
        df2 = df.drop(columns = [1,2,3,4,5,6,])
        df2_header = df2.iloc[0] #promote the first row to headers 
        df2= df2[1:]
        df2.columns = df2_header
        df = pd.concat([horse,age_color_sex,col2,col3,df2], axis=1)#concatenate the expanded data
    else:
        col2 = df.iloc[:,1]# remove the second column to expand
        col2 = col2.str.split(pat = '\r',expand = True)
        dds = col2[2].str.split(pat = ',',expand = True) # split dam and dams sire into separate columns
        dds_header = dds.iloc[0]# Save first row for headers
        dds=dds[1:]
        dds.columns = dds_header
        horse_age_color_sex = col2.iloc[:,0].str.split(pat = '(', expand = True)
        col2 = col2.drop(columns = [0,2])
        horse = horse_age_color_sex.drop(columns = 1)
        horse_head = ['Horse']
        horse=horse[1:]
        horse.columns = horse_head
        age_color_sex = horse_age_color_sex[1]#further break up column 2
        age_color_sex = age_color_sex.str.split(pat='-', expand = True)
        age_color_sex = age_color_sex[1:]
        acs_header = ['Age', 'Color', 'Sex']
        age_color_sex.columns = acs_header
       
        #replace any erroneous sex data that is be in the color column
        age_color_sex['Sex']=np.where(age_color_sex['Color'].isin(sex_list), age_color_sex['Color'], age_color_sex['Sex'])
      
        age_color_sex = age_color_sex.replace(to_replace = sex_dict)
        
        col2_header = col2.iloc[0]
        col2.columns = col2_header
        col2 = col2[1:]
        col3 = (df.iloc[:,2]).str.split(pat='\r', expand = True) # Expand column 3
        col3_header = ['Rider']
       
        # rider information was stored in the owner column in 2019 if the owner rode the horse, moving to the rider column
        col3[2] = np.where(col3.iloc[:,2].str.contains(pat = ','), col3.iloc[:,0],col3[2]) #moves owner to rider column if address carried over
        col3[2] = np.where(col3.iloc[:,2].isnull(), col3.iloc[:,0],col3[2]) #moves owner to rider column if there is no data in rider column
        
        col3 = col3.drop(columns = [0,1])
        col3.columns = col3_header
        col3=col3[1:]        
        df2 = df.drop(columns = [1,2,3,4,5,6,])
        df2_header = df2.iloc[0] #promote the first row to headers 
        df2= df2[1:]
        df2.columns = df2_header
        df = pd.concat([horse,age_color_sex,col2,col3,df2], axis=1)#concatenate the expanded data
   
    #remove header information carried over from converting each page of PDF files to CSV
    df.drop(df.loc[df['Sire']=='Sire'].index, inplace=True)
    # reset the index then rename index as the round results for that file
    df.reset_index(inplace=True)
       #rename index from 
    f = file
    fvals = f.split(sep = '.')
    fvals = fvals[0]
    fvals=fvals.split(sep = '-')
    indexstring = ' '.join(fvals[3:])
    indexstring = indexstring + ' Place'
    df = df.rename(columns = {'index':indexstring})

    return df

### Read in the files of interest and merge them. ###
<p> Formatting and reporting styles of results each year varies and results in slightly different processing order and methods of merging the files.  Some repetition of code was necessary give </p>

In [16]:
'''
The '#' did not represent a unique identifier for each horse in 2018, but the placing in any particular event or round.
The  # was dropped and files are merged.  
'''  
go1_18 = expandcolumns_goround('PB-Futurity-2018-1st-Go.csv').drop(columns = ['#'])
go2_18 = expandcolumns_goround('PB-Futurity-2018-2nd-Go.csv').drop(columns = ['#'])
pbf_18 = pd.merge(go1_18,go2_18, how = "outer")

#Eddie S nson is not a unique sire... Lets change that back to Eddie StinJson
replace = {'Eddie S nson':'Eddie Stinson','BHR Frenchie Socks':'BHR Frenchies Socks'}
pbf_18.replace(to_replace = replace, inplace = True)

#format data types as needed
pbf_18 = pbf_18.astype({'Horse' : 'string', 'Age':'float64', 'Color' : 'string', 'Sex' : 'string', 'Sire' : 'string', 'Dam' : 'string',
       "Dam's Sire":'string', 'Rider' : 'string', '1st Go': 'float64', '2nd Go': 'float64'})

0        The Goodbye Lane
1         Blazin Jetolena
2        The Goodbye Lane
3         Blazin Jetolena
4        The Goodbye Lane
             ...         
72       Pc Redwood Manny
73           Dash Ta Fame
74       Dats A Frenchman
75          Eddie Stinson
76    Bhr Frenchies Socks
Name: Sire, Length: 77, dtype: string

In [12]:
# from 2019 onward the # was a unique identifier which allowed me to merge with the unique identifier. 


go1_19 = expandcolumns_goround('PB-Futurity-2019-1st-Go.csv')
go2_19 = expandcolumns_goround('PB-Futurity-2019-2nd-Go.csv').drop(columns = ['Horse', 'Age', 'Color', 'Sex', 'Sire', 'Rider'])
pbf_19 = pd.merge(go1_19,go2_19, how ="outer") 


go1_20 = expandcolumns_goround('PB-Futurity-2020-1st-Go.csv')
go2_20 = expandcolumns_goround('PB-Futurity-2020-2nd-Go.csv').drop(columns = ['Horse', 'Age', 'Color', 'Sex','Rider', 'Sire', 'Dam', "Dam's Sire"])
pbf_20 = pd.merge(go1_20,go2_20, on ='#').drop(columns = ['None_x','None_y'])

#change time, placing and year to integers
pbf_19 = pbf_19.astype({'Horse' : 'string', 'Age':'int64', 'Color' : 'string', 'Sex' : 'string', 'Sire' : 'string',
        'Rider' : 'string', '1st Go': 'float64', '2nd Go': 'float64'})
pbf_20 = pbf_20.astype({'Horse' : 'string', 'Age':'float64', 'Color' : 'string', 'Sex' : 'string', 'Sire' : 'string', 'Dam' : 'string',
       "Dam's Sire":'string', 'Rider' : 'string', '1st Go': 'float64', '2nd Go': 'float64'})


In [15]:
# Write each years results to a csv to be called as needed

pbf_18.to_csv('./CSV_Analysis/2018_Futurity')
pbf_19.to_csv('./CSV_Analysis/2019_Futurity')
pbf_20.to_csv('./CSV_Analysis/2020_Futurity')